<a href="https://colab.research.google.com/github/ghost1998/Data/blob/master/TimeSeriesinKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Method overview : **

For this project, I have implemented the below methods
1.  LSTM model
2. Linear Regression
3. Linear Least Squares
4. Support Vector machine Regressor
5. Random Forest Regressor

** LSTM Approach **

I used an ensemble of single layered LSTMs without attention. LSTMs are a special kind of RNNs which are capable of rmembering long term dependencies. The input consists of lag variables, output consists future variables. 

  
Lag variables at a moment are the values in last n time stamps. Future variables are the values in the next immediate m future timesteps. 

Each LSTM takes the n lag values of X and Y  (total 2*n variables ) and predicts  m future values of X and Y (2*m variables). 

The returns are calculated as 60th future step value - current value. 

From each neural network, we get one returns value. The final answer is the median of these values. Median is  selected because it is most robust to outliers. 

** Rest of the Models **

All the other models use returns generated from last n time stamps as features

ie, t[i th second] -t[i -600 th second] for both xprice and yprice

And they predict the returns at that time step

ie t[i+60 th second] - t[i]

In [68]:
import numpy as np
import pandas as pd
from pandas import read_csv
from matplotlib import pyplot
import matplotlib.pyplot as plt
# from pandas.tools.plotting import autocorrelation_plot
np.nan_to_num(0)
import statsmodels
# import statsmodels.api as sm
# from statsmodels.tsa.stattools import coint, adfuller
from scipy.signal import medfilt
import random

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.svm import SVR
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.neighbors import KNeighborsRegressor
# import xgboost as xgb

from google.colab import drive
drive.mount('/content/gdrive')
# !pip install pyramid-arima
# from pyramid.arima import auto_arima
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, GRU
from keras import optimizers

def fillNan(arr):
    mask = np.isnan(arr)
    idx = np.where(~mask,np.arange(mask.shape[1]),0)
    np.maximum.accumulate(idx,axis=1, out=idx)
    arr[mask] = arr[np.nonzero(mask)[0], idx[mask]]
    arr[np.isnan(arr)] = 0
    return arr

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).



Prepare the data for the neural network. Matrix X contains historical data. y contains future data.

In [0]:
def preparedata(series, lags, steps):
  
  X1 = pd.DataFrame()
  Y1 = pd.DataFrame()
  X2 = pd.DataFrame()
  Y2 = pd.DataFrame()
  for i in range(0, lags+1):
    X1["xlag_{}".format(i)] = series.xprice.shift(i)
    X2["ylag_{}".format(i)] = series.yprice.shift(i)
        
  for i in range(1, steps+1):
    Y1["xstep_{}".format(i)] = series.xprice.shift(-i)    
    Y2["ystep_{}".format(i)] = series.yprice.shift(-i)
  
  X = np.hstack((fillNan(X1.values), fillNan(X2.values)))
  Y = np.hstack((fillNan(Y1.values), fillNan(Y2.values)))
  X = X.reshape(X.shape[0], 1, X.shape[1])
  return X, Y

**Network definition : ** 

Define the LSTM model. We use adam optimizer.

In [0]:
def getlstmmodel( lags= 40,steps = 80,  n_batch= 1000):
  print(n_batch)
  n_neurons = 50
  n_cut = 8000
  model = Sequential()
  model.add(LSTM(n_neurons, batch_input_shape=(n_batch, 1, 2+ 2*lags), stateful=True, dropout = 0.3, recurrent_dropout = 0.1))
  model.add(Dense(2*steps))
  opt = optimizers.SGD(lr=0.005, decay=1e-6)
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

def getmodels(num_models=7, lags= 40,steps = 80):
  return [getlstmmodel(lags, steps) for i in range(num_models)]

**Training : ** 

Here we had to make a decision whether to train all the networks on the same data or to split the data. The former worked slightly better. So the code corresponsing to the later is commented out.

In [0]:
def trainall(models,X, Y,transform, n_batch = 1000,n_cut = 8000, max_epochs = 2000):
  num_slices = 5 + 2*(len(models) - 1)
  for idx,model in enumerate(models):
#     p1 = X.shape[0] * ((1+2*idx - 1)/num_slices)
#     p2 = X.shape[0] * ((4+2*idx)/num_slices)
#     p3 = X.shape[0] * ((5+2*idx)/num_slices)
    p1 = X.shape[0] * (0)
    p2 = X.shape[0] * (0.7)
    p3 = X.shape[0] * (1)
    Xtrain = X[int(p1) : int(p2)]
    Xtest = X[int(p2) : int(p3)]
    Ytrain = Y[int(p1) : int(p2)]
    Ytest = Y[int(p2) : int(p3)]    
    trainlstm(model,Xtrain, Ytrain, Xtest, Ytest,transform, n_batch,n_cut, max_epochs )
 

def trainlstm (model,Xtrain, Ytrain, Xtest, Ytest,transform, n_batch=1000,n_cut =8000,max_epochs=2000):
  print("------------------------------------")
  Xtest = Xtest[: n_batch  * int( (Xtest.shape[0]/n_batch) )]
  Ytest = Ytest[ : n_batch  * int( (Ytest.shape[0]/n_batch) )]
  
  for i in range(max_epochs):
    print("Epoch " + str(i+1) + "/" + str(max_epochs) )
    if(i%10 == 0):
      pred = model.predict(Xtest, batch_size=n_batch)
      e1 = np.sqrt(np.mean(np.square(pred- Ytest)))
      print("Test Error : " + str(e1))
      predoriginal = (pred + 1) * (transform['max']- transform['min'])/2 + transform['min']
      testoriginal = (Ytest + 1) * (transform['max']- transform['min'])/2 + transform['min']
      t1 = (predoriginal[:, 139])
      t2 = (testoriginal[:, 139])
      e1 = np.sqrt(np.mean(np.square(t1- t2)) )
      e2 = r2_score(t1, t2)
      print("RMS Score: " + str(e1))
      print("R2 Score: " + str(e2))
      if(e1<0.08 and e2>0.82):
        break
      if(e1<0.9 and e2>0.9):
        break
    
      
      
    st = int(random.random() * (Xtrain.shape[0] - n_cut -2))
    ed = st+n_cut
    model.fit(Xtrain[st:ed], Ytrain[st:ed], epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
    model.reset_states()


The Function LSTMmodelEstimate() calles the above functions and preprocesses the data, defines the model(s), trains the model(s) on the preprocessed data. 

It takes arguments the csv file and number of models we wish to ensemble.


In [0]:
def LSTMmodelEstimate(filename, num_models = 3):
  print(filename)
  dataset = read_csv(filename, header=0,  index_col=0, squeeze=True)
  dataset.index = pd.to_datetime(dataset.index, unit='ms')
  originalseries = pd.DataFrame()
  originalseries['xprice']  = (dataset.xprice) 
  originalseries['yprice']  = (dataset.yprice)
  series = (2*(originalseries - np.min(originalseries.values))/(np.max(originalseries.values) - np.min(originalseries.values))) - 1
  transform = {}
  transform['min'] = np.min(originalseries.values)
  transform['max'] = np.max(originalseries.values)
  lags, steps = (40, 80)
  n_batch = 1000
  n_cut = 8000
  max_epochs = 5000
  X, Y = preparedata(series, lags, steps)
  models = getmodels(num_models, lags, steps)
  trainall(models,X,Y,transform, n_batch , n_cut,  max_epochs)
  return models, transform

The function getclassifier takes the Training data and the method name ie which classifier algorithm we wish to use, trains the classifier and returns the appropriate classifier object.

In [0]:
def getclassifier(Atrain,ytrain, method ):


    clf = None
    
    if(method == 'RandomForestRegressor'):
      clf = RandomForestRegressor( random_state=0, n_estimators=20)
      clf.fit(Atrain, ytrain)
    
    elif(method == 'SVR'):  
      clf = SVR()
      clf.fit(Atrain, ytrain)
      
    elif(method is 'LinearRegression'):
      clf = LinearRegression()
      clf.fit(Atrain, ytrain)
      
    elif(method is 'LeastSquares'):
      X = np.linalg.lstsq(Atrain, ytrain, rcond=None)[0]
      return X
      
    elif(method == 'xgBoost'):
      xgdmat = xgb.DMatrix(data=Atrain,label=ytrain)
      params={'objective':'reg:linear','max_depth':25, 'n_estimators' : 250}
      clf=xgb.train(params,xgdmat) 
    else:
      print("Cannot recognise the method " + str(method))
      return
    
    return clf


The Function CLFmodelEstimate() calles the above functions and preprocesses the data, defines the classifier, trains the classifier on the preprocessed data.

It takes arguments the csv file and method ie which classifier algorithm we wish to use.

In [0]:
def clfmodelEstimate(filename, method = 'LinearRegression'):
    dataset = read_csv(filename, header=0,  index_col=0, squeeze=True)
    dataset.index = pd.to_datetime(dataset.index, unit='ms')
    dataset['xreturns'] =  dataset['xprice'].shift(-60) - dataset['xprice']
  
    series = pd.DataFrame()
    series['xprice'] = dataset['xprice'] - dataset['xprice'].shift(60)
    series['yprice'] = dataset['yprice'] - dataset['yprice'].shift(60)
    lags = 150

  
  
    print("Number of lags : " + str(lags))
    lagdata = pd.DataFrame()

    for i in range(0, lags+1):
        lagdata["ylag_{}".format(i)] = series.yprice.shift(i)
        lagdata["xlag_{}".format(i)] = series.xprice.shift(i)

    yval = dataset['returns']
    ytrain = yval.values

    clf = None

    A = lagdata.values
    A = np.nan_to_num(A)
    Atrain = A
    
    clf = getclassifier(Atrain,ytrain, method)
    return clf
  

This is the final function which calls all the above functions when needed. 

It takes name of the csv file and the method ie ie which  algorithm we wish to use as arguments.

It returns parameter dictionary which has all the information for inference.

In [0]:
def modelEstimate(filename, method = 'LinearRegression'):
  params = {}
  params['method'] = method
  if(method == 'LSTM'):
    models, transform = LSTMmodelEstimate(filename)
    params['models'] = models
    params['transform'] = transform
  else:
    params['clf'] = clfmodelEstimate(filename, method)
  return params
    

Call the modelEstimate function

In [56]:
filename = '/content/gdrive/My Drive/data.csv'
params = modelEstimate(filename, method='LinearRegression')

Number of lags : 20


In [57]:
params

{'clf': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False), 'method': 'LinearRegression'}

**Inference : **


In [0]:
def predict(models, X, Y, n_batch):
  pad = np.zeros(((n_batch - len(X)%n_batch  ), X[0].shape[0], X[0].shape[1]))
  X_new = np.concatenate((X, pad), axis=0)
  stck = []
  for i in range(len(models)):
    pr = models[i].predict(X_new, batch_size=n_batch)
    stck.append(pr)
  stacked = np.stack(stck)
  Ypredwithpad = np.median(stacked, axis = 0)
  Ypred = Ypredwithpad[:X.shape[0]]
#   print(mean_squared_error(Y, Ypred))
  return Ypred

In [0]:
def LSTMmodelForecast(filename, params):
  print(filename)
  dataset = read_csv(filename, header=0,  index_col=0, squeeze=True)
  dataset.index = pd.to_datetime(dataset.index, unit='ms')
  originalseries = pd.DataFrame()
  originalseries['xprice']  = dataset.xprice 
  originalseries['yprice']  = dataset.yprice 
  
  transform = params['transform']
  models = params['models']
  series = (2*(originalseries - transform['min'])/(transform['max'] - transform['min'])) - 1
  lags, steps = (40, 80)
  n_batch = 1000
  n_cut = 8000
  max_epochs = 200
  X, Y = preparedata(series, lags, steps)
  Ypred = predict(models, X, Y, n_batch)
  returns = ((Ypred[:, 139]+1)* (transform['max']- transform['min'])/2 + transform['min'] ) - ((X[:, 0, lags+1] + 1) * (transform['max']- transform['min'])/2 + transform['min'])
  return returns

In [0]:
def predictclassifier(Atest,clf, method):

    predtest = None
    if(method == 'RandomForestRegressor'):
      predtest = clf.predict(Atest)
    
    elif(method == 'SVR'):  
      predtest = clf.predict(Atest)
      
    elif(method is 'LinearRegression'):
      predtest = clf.predict(Atest)
      
    elif(method is 'LeastSquares'):
      predtest = np.matmul(Atest, clf)
      
    elif(method == 'xgBoost'):


      mat=xgb.DMatrix(Atest)
      predtest=clf.predict(mat)
    else:
      print("Cannot recognise the method " + str(method))
      return
    
    return predtest


In [0]:
def clfmodelForecast(filename, params):
    dataset = read_csv(filename, header=0,  index_col=0, squeeze=True)
    dataset.index = pd.to_datetime(dataset.index, unit='ms')
    dataset['xreturns'] =  dataset['xprice'].shift(-60) - dataset['xprice']
  
    series = pd.DataFrame()
    series['xprice'] = dataset['xprice'] - dataset['xprice'].shift(60)
    series['yprice'] = dataset['yprice'] - dataset['yprice'].shift(60)
    lags = 20

  
  
    print("Number of lags : " + str(lags))
    lagdata = pd.DataFrame()

    for i in range(0, lags+1):
        lagdata["ylag_{}".format(i)] = series.yprice.shift(i)
        lagdata["xlag_{}".format(i)] = series.xprice.shift(i)

    yval = dataset['returns']
    ytest = yval.values

    clf = None

    A = lagdata.values
    A = np.nan_to_num(A)
    Atest = A
    
    method = params['method']
    clf = params['clf']
    
    return predictclassifier(Atest,clf, method)
    
  


The function modelForecast() function takes the filename and preproccesses the data appropriately and then calls predict() function.

If the method is LSTM,  it calls LSTMmodelForecast() which runs all the models and takes the median. 

Else it calls clfmodelForecast() which calls their respective predict functions and infers the data.

It returns the returns at all the time points



In [0]:
def modelForecast(filename, params):
  if(params['method'] is 'LSTM'):
    return LSTMmodelForecast(filename, params)
  return clfmodelForecast(filename, params)
  

In [66]:
filename = '/content/gdrive/My Drive/data.csv'
returns = modelForecast(filename, params)

Number of lags : 20



**Winner** : Linear Regression seemed to perform the best out of all on validation data. (with an RMS of 0.017)

Also I have avoided using packages like fbProphet, AutoArima, ARIMAX, VARMAX, fbprophet because 
1. They  are a single line pieces of code. So it is naturally I assumed they are prohibitted
2. Besides that they are very slow and have to be fit at each time step which will take very much time

**Other Methods and Baselines : **

1. Ridge and Lasso Regression
2. Seq to Seq model
3. Seq to Seq Models with attention

** Feature Selection : **
1. At each timestamp lags and future values are used for LSM.
2. Median features (ie median over last 5 min, 10 min, 20 min, 30 min, 60 min, 120 min, 240 min, 480 min) are experimented with and dropped because they give little to no boost in performance with enormous toll on performance. (In rerospection, approximate median kernals could have been used)
3. immediate differences (x[t] - x[t-1]) were experimented with. Though these features pass Dickey-Fuller test and should work better, the LSTMs seemed to perform worse. 
4. For the rest of the classifiers returns generated from last n time stamps are used as features
 ie, t[i th second] -t[i -600 th second] for both xprice and yprice